In [6]:
import pandas as pd
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

import time

from datetime import datetime
from datetime import date


from openpyxl import load_workbook

In [2]:
urls = ['https://www.boerse-frankfurt.de/bond/xs1892141620-rumaenien-republik-2-875-18-29/price-history/historical-prices-and-volumes', 
       'https://www.boerse-frankfurt.de/bond/usp01012bx31-el-salvador-republik-8-625-17-29/price-history/historical-prices-and-volumes',
       'https://www.boerse-frankfurt.de/bond/us040114hs26-argentinien-republik-0-5-20-30/price-history/historical-prices-and-volumes',
       'https://www.boerse-frankfurt.de/bond/xs2214237807-ecuador-republik-5-5-20-30/price-history/historical-prices-and-volumes',
       'https://www.boerse-frankfurt.de/bond/xs1297557412-ghana-republic-of-10-75-15-30/price-history/historical-prices-and-volumes',
       'https://www.boerse-frankfurt.de/bond/xs1910827887-nigeria-bundesrepublik-8-747-18-31/price-history/historical-prices-and-volumes',]

In [5]:
url = 'https://www.boerse-frankfurt.de/bond/us040114hs26-argentinien-republik-0-5-20-30/price-history/historical-prices-and-volumes'

In [3]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True

def get_row_data(elem):
    date = elem.find(name = 'td', attrs = {'class' : 'widget-table-cell'}).text
    percentages = elem.find_all(name = 'td', attrs = {'class' : 'widget-table-cell text-right'})
    volumes = elem.find_all(name = 'td', attrs = {'class' : 'widget-table-cell text-right ng-star-inserted'})
    return [date, percentages[0].text, percentages[1].text, percentages[2].text, percentages[3].text, volumes[0].text, volumes[1].text]

In [7]:
first = True
filepath = 'boerse/boerse-' + str(date.today())+'.xlsx'
for url in urls:
    df = pd.DataFrame(columns = ['Date', 'Open', 'Close', 'High', 'Low', 'Volume €', 'Volume nominal'])

    start = time.time()

    options = Options()
    # options.add_experimental_option("excludeSwitches", ["enable-automation"])
    # options.add_experimental_option('useAutomationExtension', False)
    # options.add_argument('--disable-blink-features=AutomationControlled')


    options.binary_location = 'C:\Program Files\Google\Chrome\Application\chrome.exe'
    driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\Program Files (x86)\chromedriver.exe')

    driver.get(url)
    try:
        action = ActionChains(driver)
    
        # Setting up the page
        xpath_100 = '/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[2]/div/app-price-history/div[2]/div/div/app-page-bar[1]/div/div[2]/button[3]'
        button_100 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_100)))
        action.click(on_element = button_100)
        action.perform()
        time.sleep(5)
    
        i = 4
        while (check_exists_by_xpath('/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[2]/div/app-price-history/div[2]/div/div/app-page-bar[1]/div/div[1]/button['+str(i + 2)+']')):
            page_source = driver.page_source
            soup = bs(page_source, 'html.parser')
            
            # Retrieving data
            li = soup.find_all(name = 'tr', attrs = {'class' : 'widget-table-row ng-star-inserted'})
            for elem in li:
                df.loc[len(df.index)] = get_row_data(elem)
    
            xpath = '/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[2]/div/app-price-history/div[2]/div/div/app-page-bar[1]/div/div[1]/button['+str(i)+']'
            next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
            action.click(on_element = next_button)
            action.perform()
            time.sleep(5)
            i += 1
        
        page_source = driver.page_source
        soup = bs(page_source, 'html.parser')
        
        li = soup.find_all(name = 'tr', attrs = {'class' : 'widget-table-row ng-star-inserted'})
        for elem in li:
            df.loc[len(df.index)] = get_row_data(elem)
    
        country = soup.find(attrs = {'class' : 'instrument-name'}).text
    except:
        print('murio')
        driver.quit()
    
    driver.quit()
    
    end = time.time()
    
    print(end - start)
    
    name = ''
    for word in country.split():
        if not (word[len(word) - 1] == ','):
            name = name + word
        else:
            name = name + word[0:(len(word) - 1)]
            break
    print(name)
    
    if first:
        first = False
        df.to_excel(filepath, sheet_name = name, index = False)
    else:
        # Generating workbook
        ExcelWorkbook = load_workbook(filepath)
         
        # Generating the writer engine
        writer = pd.ExcelWriter(filepath, engine = 'openpyxl')
         
        # Assigning the workbook to the writer engine
        writer.book = ExcelWorkbook
         
        # Adding the DataFrames to the excel as a new sheet
        df.to_excel(writer, sheet_name = name)
        writer.save()
        writer.close()


<ipython-input-7-f8209d5d063f>:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\Program Files (x86)\chromedriver.exe')
<ipython-input-7-f8209d5d063f>:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\Program Files (x86)\chromedriver.exe')


29.279595136642456
Rumänien
28.518657445907593
ElSalvador
28.438639163970947
Argentinien
29.111467838287354
Ecuador
28.7118182182312
Ghana
28.746888637542725
Nigeria
